In [116]:
import pandas as pd

df = pd.read_csv('software_requirements_extended.csv')
df.head()

,Type,Requirement
0,PE,The system shall refresh the display every 60 ...
1,LF,The application shall match the color of the s...
2,US,If projected the data must be readable. On ...
3,A,The product shall be available during normal ...
4,US,If projected the data must be understandable...


In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977 entries, 0 to 976
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Type         977 non-null    object
 1   Requirement  977 non-null    object
dtypes: object(2)
memory usage: 15.4+ KB


In [118]:
import plotly.express as px

# draw a pie chart of the different sentiments of the posts
fig = px.pie(df, names='Type', title ='Pie of types of requirements')
fig.show()

data set is small and not balanced to write a model to predict all the types of requirements. but its enough and balanced to predict if type is functional or non-functional

In [119]:
df['Type'] = df['Type'].apply(lambda x: 1 if x == 'FR' or x == 'F' else 0)
fig = px.pie(df, names='Type', title ='Pie of types of requirements')
fig.show()

In [120]:
import re
import nltk  #natural language processing
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import *

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nickbres/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [121]:
pattern  = re.compile(r"[^a-zA-Z0-9]")
def req_to_words(req):
    ''' Convert tweet text into a sequence of words '''
    # convert to lower case
    text = req.lower()
    # remove non letters
    text = re.sub(pattern, " ", text)
    # tokenize
    words = text.split()
    # remove stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    # apply stemming
    words = [PorterStemmer().stem(w) for w in words]
    # return list
    return words

print("\nOriginal tweet ->", df['Requirement'][0])
print("\nProcessed tweet ->", req_to_words(df['Requirement'][0]))


Original tweet -> The system shall refresh the display every 60 seconds.

Processed tweet -> ['system', 'shall', 'refresh', 'display', 'everi', '60', 'second']


In [122]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

## Without preprocessing

In [123]:
X = df['Requirement']
y = df['Type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [124]:
vectorizer = TfidfVectorizer(analyzer=req_to_words)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [125]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [126]:
# 3. Prediction and Evaluation
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Detailed performance report
print(classification_report(y_test, y_pred))

Accuracy: 85.42%
              precision    recall  f1-score   support

           0       0.85      0.82      0.84       176
           1       0.86      0.88      0.87       215

    accuracy                           0.85       391
   macro avg       0.85      0.85      0.85       391
weighted avg       0.85      0.85      0.85       391


## With preprocessing

In [127]:
y = df['Type'] # Target
X = [' '.join(req_to_words(req)) for req in df['Requirement']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [128]:
vectorizer2 = TfidfVectorizer(analyzer=req_to_words)
X_train = vectorizer2.fit_transform(X_train)
X_test = vectorizer2.transform(X_test)

In [129]:
model2 = LogisticRegression()
model2.fit(X_train, y_train)

LogisticRegression()

In [130]:
# 3. Prediction and Evaluation
y_pred = model2.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Detailed performance report
print(classification_report(y_test, y_pred))

Accuracy: 85.42%
              precision    recall  f1-score   support

           0       0.85      0.82      0.84       176
           1       0.86      0.88      0.87       215

    accuracy                           0.85       391
   macro avg       0.85      0.85      0.85       391
weighted avg       0.85      0.85      0.85       391


In [131]:
def predict_req(req,expected,model,vectorizer):
    df = pd.DataFrame([req], columns=['Requirement'])
    X = [' '.join(req_to_words(req)) for req in df['Requirement']]
    X = vectorizer.transform(X)
    
    prediction = model.predict(X)
    predicted_label = 'Functional' if prediction[0] == 1 else 'Non-Functional'
    print (f'{req} -> {predicted_label}')
    if predicted_label == expected:
        print(f'{expected} -> Success!')
    else:
        print(f'{expected} -> Fail!')
    

In [132]:
req_example_1 = "The system shall provide a login form that accepts a username and password."
expected_label_1 = 'Functional'
predict_req(req_example_1, expected_label_1,model, vectorizer)

req_example_2 = "The application must ensure that response times are less than 2 seconds under normal load conditions."
expected_label_2 = 'Non-Functional'
predict_req(req_example_2, expected_label_2,model, vectorizer)

req_example_1_2 = "The system shall provide a login form that accepts a username and password."
expected_label_1_2 = 'Functional'
predict_req(req_example_1_2, expected_label_1_2,model2, vectorizer2)

req_example_2_2 = "The application must ensure that response times are less than 2 seconds under normal load conditions."
expected_label_2_2 = 'Non-Functional'
predict_req(req_example_2_2, expected_label_2_2,model2, vectorizer2)


The system shall provide a login form that accepts a username and password. -> Functional
Functional -> Success!
The application must ensure that response times are less than 2 seconds under normal load conditions. -> Non-Functional
Non-Functional -> Success!
The system shall provide a login form that accepts a username and password. -> Functional
Functional -> Success!
The application must ensure that response times are less than 2 seconds under normal load conditions. -> Non-Functional
Non-Functional -> Success!
